In [1]:
from openbb import obb
import polars as pl
import datetime as dt
import infomeasure as im
price = obb.crypto.price.historical(symbol='BTCUSD,ETHUSD', start_date=dt.date(2020, 1, 1))
price = pl.DataFrame(map(lambda d: d.model_dump(), price.results)).pivot(on='symbol', index='date')
price.head()

date,open_BTC-USD,open_ETH-USD,high_BTC-USD,high_ETH-USD,low_BTC-USD,low_ETH-USD,close_BTC-USD,close_ETH-USD,volume_BTC-USD,volume_ETH-USD,vwap_BTC-USD,vwap_ETH-USD
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,null,null
2020-01-01,7194.89209,129.630661,7254.330566,132.835358,7174.944336,129.198288,7200.174316,130.802002,1.8566e10,7.9352e9,null,null
2020-01-02,7202.55127,130.820038,7212.155273,130.820038,6935.27002,126.95491,6985.470215,127.410179,2.0802e10,8.0327e9,null,null
2020-01-03,6984.428711,127.411263,7413.715332,134.554016,6914.996094,126.490021,7344.884277,134.171707,2.8111e10,1.0477e10,null,null
2020-01-04,7345.375488,134.168518,7427.385742,136.052719,7309.51416,133.040558,7410.656738,135.069366,1.8444e10,7.4309e9,null,null
2020-01-05,7410.45166,135.072098,7544.49707,139.410202,7400.535645,135.045624,7411.317383,136.276779,1.9725e10,7.5267e9,null,null


# WHat is the best pearson correlation ?
Pearson is a very naive approach but still our baseline

In [2]:
a, b = pl.col("open_BTC-USD").pct_change().fill_null(0.0), pl.col("open_ETH-USD").pct_change().fill_null(0.0)
price.select(
    #x=pl.concat_arr([chatterjee_corr(a.shift(i), b) for i in range(1, 10)]), 
    y=pl.concat_arr([pl.corr(a.shift(i), b) for i in range(1, 10)]),
    #z=pl.concat_arr([chatterjeexi(a.shift(i), b).statistic for i in range(1, 10)])
).row(0)

([-0.059173672354839245,
  0.046484146894598004,
  0.006653997702862518,
  0.03608722362680497,
  -0.0024201838023710166,
  0.0031423510855921153,
  -0.027702729333123833,
  -0.022497152222202967,
  0.030570469078559644],)

In [178]:
[
    im.transfer_entropy(
        price.get_column(a.meta.output_name()).pct_change().drop_nulls(),
        price.get_column(b.meta.output_name()).pct_change().drop_nulls(),
        approach='ordinal',
        embedding_dim=3,
        prop_time=i,
        ) for i in range(1, 15)
]

[np.float64(0.029392302501231704),
 np.float64(0.0208056918171481),
 np.float64(0.011771069610498038),
 np.float64(0.011345068148582459),
 np.float64(0.013858167867168557),
 np.float64(0.01613016698556156),
 np.float64(0.01967587843790836),
 np.float64(0.015397100468604546),
 np.float64(0.010679148003875791),
 np.float64(0.010115855742073155),
 np.float64(0.013495316466123335),
 np.float64(0.011959975156428326),
 np.float64(0.012436554092852471),
 np.float64(0.011420646381283426)]

In [183]:
[
    im.mutual_information(
        price.get_column(a.meta.output_name()).shift(i).drop_nulls(),
        price.get_column(b.meta.output_name()).drop_nulls().slice(i),
        approach='ordinal',
        embedding_dim=3,
        #prop_time=i,
        ) for i in range(1, 15)
]

[np.float64(0.25578938461738077),
 np.float64(0.009277801587318862),
 np.float64(0.008306391155028833),
 np.float64(0.006532856571341519),
 np.float64(0.007839100932980077),
 np.float64(0.004055099177106107),
 np.float64(0.006440962204480774),
 np.float64(0.0063353818959324334),
 np.float64(0.005902383374356661),
 np.float64(0.006040709038773462),
 np.float64(0.005280717361537499),
 np.float64(0.003887524283383903),
 np.float64(0.00752529956345855),
 np.float64(0.01026017254978883)]

Let us
 - compute transfer entropy on a rolling window
 - do that for every possible lag value
 - take the arg_max() yielding the best TE

In [52]:
def optimal_alignment(source: pl.Expr, target: pl.Expr, lags=range(2, 15), period='1q', offset="-45d", min_block_length=10):
    assert lags.step > 0
    wrap = pl.concat_arr(pl.struct(source, target).rolling('date', period=period, offset=offset).list.eval(pl.element().struct.unnest()))
    fn = lambda lag: wrap.map_elements(
        lambda x: im.transfer_entropy(x[0], x[1], approach='ordinal', embedding_dim=3, prop_time=lag),
        return_dtype=pl.Float64
    )
    # caompute TE for all lags; small penalty on big lags; i.e we only pick them when TE is much bigger
    optimal_lag = pl.concat_list([fn(lag) * (1 - (lag / (lags.stop * 1.2))) for lag in lags]).list.arg_max().add(1)
    return optimal_lag.rolling_median(min_block_length).cast(pl.UInt8).backward_fill()

final = price.select(
    'date', 
    optimal_alignment_change=optimal_alignment(pl.col("open_BTC-USD").pct_change().fill_null(0.0), pl.col("open_ETH-USD").pct_change().fill_null(0.0)),
    #optimal_alignment_raw=optimal_alignment(pl.col("open_BTC-USD").pct_change().fill_null(0.0), pl.col("open_ETH-USD"))
    
)
final
#tmp

date,optimal_alignment_change
date,u8
2020-01-01,3
2020-01-02,3
2020-01-03,3
2020-01-04,3
2020-01-05,3
…,…
2025-10-01,2
2025-10-02,2
2025-10-03,2


### TODO
 - test if TE on aligned-source to target yields better TE than the best fix lag 
 - test if this TE is resistant to 